# CVPR2017 papers web crawler

In [102]:
### Import and initialize main page
import urllib2
req = urllib2.Request('http://openaccess.thecvf.com/CVPR2017.py')
response = urllib2.urlopen(req)
page = response.read()

In [103]:
### Crawl from main page
import re
myItems = re.findall('<dt class="ptitle">(.*?)</div>\n</div>\n</dd>',page,re.S)

for item in myItems:
    mainPages = re.findall('<br><a href="(.*?)">',page,re.S)
    pdfPages = re.findall('</dd>\n<dd>\n.<a href="content_cvpr_2017/papers/(.*?)">pdf</a>',page,re.S)
    firstAttempAuthors = re.findall('author = {(.*?)},<br>',page,re.S)
    titles = re.findall('<br>\ntitle = {(.*?)},<br>',page,re.S)
        
#print 'size of authors:', len(authors), '\n'
#print 'size of titles:' , len(titles), '\n'
#print 'size of mainPages:' , len(mainPages), '\n'
#print 'size of pdfPages:' , len(pdfPages), '\n'
    
#    print authors[781]
#    print titles[781]
#    print mainPages
print 'number of all papers to process: ', len(pdfPages)
    #print pdfPages[0]
    #return pdfPages[0]


def getContext(url):  
    text =urllib2.urlopen(url).read()  
    return text  
  
def StoreContext(url):  
    content  = getContext(url)  
    filename = url[52:]  
    open(filename, 'w').write(content) 

#StoreContext(url0)

number of all papers to process:  783


## 1. Get author names on each paper

2 methods on getting the author names:
    1. get from the main page, under each title name;
    2. get from the paper's own page,under title name.
    
The problem with the second method is that some pages are no longer valid now. So I added a judgement part to test if the page is valid. Same judgement is also used when accessing the pdf pages.

In [132]:
# check for invalid pages
from urllib2 import URLError
result_url=[]  
allAuthors = []
need_second_attemp = []

for i in range(len(mainPages)):
    url = 'http://openaccess.thecvf.com/' + mainPages[i]
    try:
        response=urllib2.urlopen(url)
    except URLError, e:
        if hasattr(e,'reason'): #stands for URLError
            print 'Error at paper number #', i, ':'
            print "can not reach a server at " + url
            result_url.append(url)
            need_second_attemp.append(i)
        else: #stands for HTTPError or other
            print 'Error at paper number #', i, ':'
            print "find http error at " + url
            result_url.append(url)
            need_second_attemp.append(i)
    else:
        #print "url is reachable!"
        reqMain = urllib2.Request(url)
        responseMain = urllib2.urlopen(reqMain)
        mainPage = responseMain.read()
        currAuthors = re.findall('<div id="authors">\n<br><b><i>(.*?)</i></b>',mainPage,re.S)[0]
        allAuthors.append(currAuthors)
        response.close()
    finally:
        pass

Error at paper number # 257 :
can not reach a server at http://openaccess.thecvf.com/content_cvpr_2017/html/Ilg_FlowNet_2.0_Evolution_CVPR_2017_paper.html
Error at paper number # 508 :
can not reach a server at http://openaccess.thecvf.com/content_cvpr_2017/html/Yuan_BigHand2.2M_Benchmark_Hand_CVPR_2017_paper.html


In [147]:
### Second attempt on the invalid pages
if len(need_second_attemp) != 0:
    newAllAuthors = []
    for item in allAuthors:
        item = str(item).split(', ')
        newAllAuthors.append(item)

    while need_second_attemp:
        index = need_second_attemp.pop(0)
        curr = str(firstAttempAuthors[index])
        curr = curr.split(' and ')
        newCurr = []
        for item in curr:
            item = str(item).split(', ')
            newItem = str(item[1] + ' ' + item[0])
            newCurr.append(newItem)
        newAllAuthors.insert(index, newCurr)
#for item in newAllAuthors:
    #print item

In [149]:
### some examples:
for i in range(10):
    print newAllAuthors[i]

['Damien Teney', 'Lingqiao Liu', 'Anton van den Hengel']
['Hao Zhao', 'Ming Lu', 'Anbang Yao', 'Yiwen Guo', 'Yurong Chen', 'Li Zhang']
['Felix Juefei-Xu', 'Vishnu Naresh Boddeti', 'Marios Savvides']
['Yagiz Aksoy', 'Tunc Ozan Aydin', 'Marc Pollefeys']
['George Trigeorgis', 'Patrick Snape', 'Iasonas Kokkinos', 'Stefanos Zafeiriou']
['James Booth', 'Epameinondas Antonakos', 'Stylianos Ploumpis', 'George Trigeorgis', 'Yannis Panagakis', 'Stefanos Zafeiriou']
['Vamsi Kiran Adhikarla', 'Marek Vinkler', 'Denis Sumin', 'Rafal K. Mantiuk', 'Karol Myszkowski', 'Hans-Peter Seidel', 'Piotr Didyk']
['Manikanta Kotaru', 'Sachin Katti']
['Kenichiro Tanaka', 'Yasuhiro Mukaigawa', 'Takuya Funatomi', 'Hiroyuki Kubo', 'Yasuyuki Matsushita', 'Yasushi Yagi']
['Philip Haeusser', 'Alexander Mordvintsev', 'Daniel Cremers']


## 2. Get instituitions information from papers' pdf links

    * used *pdfminer* to convert a pdf url into text;
    * then used *urllib2* to get all the content before "Abstract";
    * deleted title, author names and emails text;
    * deleted characters that are not alphabets. (to get rid of quotation marks and such)

But the current model is still not working so well with:

    1. Unsual strcture:
    
    some papers may have figures and figure labels before the abstract, which will be interpreted as "institution information" in this model. Like this paper here: http://openaccess.thecvf.com/content_cvpr_2017/papers/Simon_Hand_Keypoint_Detection_CVPR_2017_paper.pdf. Which will be detected as an institution.
        
    2. Unexpected characters:
    
    some author names are not unified between paper's pdf page and paper's own page (often happens when an author is using charactors that are not included in English). For example, for this page: Towards a Quality Metric for Dense Light Fields. On its own page (http://openaccess.thecvf.com/content_cvpr_2017/html/Adhikarla_Towards_a_Quality_CVPR_2017_paper.html)(and also on the main page) the 4th author's name is listed *"Rafal K"*. However, in the pdf version (http://openaccess.thecvf.com/content_cvpr_2017/papers/Adhikarla_Towards_a_Quality_CVPR_2017_paper.pdf), this author's name is presented as *"Rafał K. Mantiuk"*. And the character *"ł"* cannot be detected as the same as the English character *"l"*. In this case, the string *"Rafał K. Mantiuk"* cannot match *"Rafal K"* (which is already acquired as one of the author names under this paper in previous step). As a result, *"Rafał K. Mantiuk"* won't be detected as an author name, won't be removed, and will be treated as an institution. 

In [137]:
# check for invalid pages (None this time)
from urllib2 import URLError
valid_cnt = 0
for i in range(len(pdfPages)):
    url = 'http://openaccess.thecvf.com/content_cvpr_2017/papers/' + pdfPages[i]
    try:
        response=urllib2.urlopen(url)
    except URLError, e:
        #print url
        if hasattr(e,'reason'): #stands for URLError
            print 'paper number #', i
            print "can not reach a server at " + url
        else: #stands for HTTPError or other
            print 'paper number #', i
            print "find http error at " + url
    else:
        valid_cnt += 1
        
    finally:
        pass

In [138]:
import urllib2
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def Pdf_Write2txt(html):
    rsrcmgr = PDFResourceManager()

    retstr = StringIO()
    laparams=LAParams()
    device = TextConverter(rsrcmgr, retstr, codec='utf-8', laparams=laparams)

    fp = StringIO(html)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    pagenos=set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=0, password="",caching=True, check_extractable=True):
        interpreter.process_page(page)
        break
    fp.close()
    device.close()
    currPage = retstr.getvalue()
    retstr.close()
    return currPage
    
def get_pdf_url(url):
    url = 'http://openaccess.thecvf.com/content_cvpr_2017/papers/' + url
    html = urllib2.urlopen(urllib2.Request(url)).read()
    return html

In [150]:
### Get institution information
allInst = []
for num_paper in range(len(pdfPages)):

    currPage = Pdf_Write2txt(get_pdf_url(pdfPages[num_paper]))
    hasAbstract = bool(re.search('abstract', currPage, re.IGNORECASE)) 

    if not hasAbstract:
        allInst.append(str('cannot detect Abstract at: ' + str(num_paper)))
    else:
        abSplitPage = re.split('abstract', currPage, flags=re.I)[0]
        
        ########
        ### transfer all to lower case, and remove all symbols
        lowerPage = abSplitPage.lower()
        lowerPage = re.sub('ﬁ', 'fi', lowerPage)
        noSymbolPage = re.sub(r'[^a-zA-Z0-9\s@,]',r'',lowerPage)

        ########
        ### remove author names
        for item in newAllAuthors[num_paper]:
            name = item.lower()
            name = re.sub(r'[^a-zA-Z0-9\s@,]',r'',name)
            #print name
            noSymbolPage = re.sub(str(name), '', noSymbolPage)

        ########
        ### remove title
        title = titles[num_paper].lower()
        title = re.sub(r'[^a-zA-Z0-9\s@,]',r'',title)
        title = re.sub(' ', '\s+', title)
        newLowerPage = re.sub(str(title), '', noSymbolPage, re.S)
        
        #########
        ### remove emails and citation marks
        newLowerPage2 = re.sub('\s.+@.+','',newLowerPage,re.S)
        newLowerPage3 = re.sub(r'[^a-zA-Z\s\-]',r'',newLowerPage2)
        emptyLineCount = 0
        
        lineSplitPage = newLowerPage3.split('\n')
        institution = []
        for line in lineSplitPage:
            line = re.sub('\s+and\s+','',line)
            line = line.strip()
            if len(line) == 0 or line == '' or line == ' ':
                emptyLineCount += 1
            else:
                line = re.sub('\r*\n*\f*\t*\v*', '', line)
                institution.append(line)
        #print institution
        allInst.append(institution)

In [151]:
### some examples:
for i in range(10):
    print allInst[i]

['australian centre for visual technologies', 'the university of adelaide']
['department of electronic engineering tsinghua university', 'cognitive computing laboratory intel labs china']
['carnegie mellon university', 'michigan state university', 'carnegie mellon university']
['yagz aksoy tunc ozan aydn', 'eth zurich', 'disney research zurich']
['imperial college london', 'imperial college london', 'university college london', 'imperial college london']
['imperial college london uk', 'amazon berlin germany', 'university of oulu finland']
['rafa k mantiuk', 'mpi informatik', 'saarland university mmci', 'the computer laboratory university of cambridge']
['stanford university']
['nara institute of sciencetechnology naist', 'osaka univerisity']
['dept of informatics tu munich', 'google inc']


## 3. Get topics from analysing papers' abstract

Used Topic modeling to get 3 topic keywords from abstract of each paper.

    * converted abstract into a matrix representation;
    * create an object for LDA model using gensim library;
    * run and train the LDA model on the term matrix;
    * get 1 topic with 3 keywords.

In [153]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

allTopics = []
for i in range(len(mainPages)):
    if i == 257 or i == 508:
        allTopics.append(titles[i])
    else:
        url = 'http://openaccess.thecvf.com/' + mainPages[i]

        #print "url is reachable!"
        reqMain = urllib2.Request(url)
        responseMain = urllib2.urlopen(reqMain)
        mainPage = responseMain.read()
        abstract = re.split('id="abstract"', mainPage, flags=re.I)[1]
        abstract = re.split('<font size="5">', abstract, flags=re.I)[0]
        abstract = re.split('</div>\s*', abstract, flags=re.I)[0]
        #print abstract
        abstract = str(abstract).split('.')
        doc_complete = []
        for item in abstract:
            newItem = re.sub(r'[^a-zA-Z0-9 @,]',r'',item)
            newItem = newItem.strip()
            if newItem != '':
                doc_complete.append(newItem)
        #print doc_complete


        doc_clean = [clean(doc).split() for doc in doc_complete]
        #print doc_clean

        # Importing Gensim
        import gensim
        from gensim import corpora
        from gensim.corpora import Dictionary

        dictionary = corpora.Dictionary(doc_clean)

        # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
        # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
        doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

        # Creating the object for LDA model using gensim library
        Lda = gensim.models.ldamodel.LdaModel

        # Running and Trainign LDA model on the document term matrix.
        ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

        topic = ldamodel.print_topics(num_topics=1, num_words=3)
        #print topic[0][1]
        allTopics.append(topic[0][1])

In [154]:
### some examples:
for i in range(10):
    print allTopics[i]

0.055*"scene" + 0.038*"accuracy" + 0.038*"question"
0.051*"method" + 0.027*"network" + 0.027*"layout"
0.042*"lbc" + 0.042*"binary" + 0.029*"cnns"
0.044*"region" + 0.030*"unknown" + 0.030*"information"
0.035*"convolutional" + 0.035*"fully" + 0.035*"method"
0.050*"facial" + 0.050*"model" + 0.038*"shape"
0.058*"quality" + 0.041*"lightfield" + 0.023*"existing"
0.040*"vr" + 0.040*"g" + 0.023*"e"
0.064*"material" + 0.052*"object" + 0.052*"camera"
0.044*"data" + 0.044*"unlabeled" + 0.031*"use"


## 4. Export data into txt files

In [111]:
fileObject = open('All_Authors.txt', 'w')
for item in newAllAuthors:
    for ip in item:
        fileObject.write(ip)
        fileObject.write(', ')
    fileObject.write('\n')
fileObject.close()

In [99]:
fileObject = open('All_Topics.txt', 'w')
for ip in allTopics:
    fileObject.write(ip)
    fileObject.write('\n')
fileObject.close()

In [144]:
fileObject = open('All_Institution.txt', 'w')
for item in allInst:
    for ip in item:
        fileObject.write(ip)
        fileObject.write(', ')
    fileObject.write('\n')
fileObject.close()